In [1]:
import time, re
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
pendant = "https://www.slader.com"
chrome_path = "/Users/DCT/Desktop/chromedriver_win32/chromedriver"
file_path = "C:/Users/DCT/Desktop/slader_textbook"
book_list = ["algorithm","calculus"] # store book name
book_url = [] #stor book url

In [3]:
import os, stat
import urllib.request

def download_img(dir_path, name, url):
    try:
        if not os.path.exists(dir_path): # make the file if not exist
            os.makedirs(dir_path)
        #suffix = "." + url.split(".")[-1]
        suffix = ".jpg"
        file_name = "{}{}{}{}".format(dir_path, os.sep, name, suffix)
        urllib.request.urlretrieve(url, file_name)
        #print(file_name)
        return True
    except:
        print("download_img error, dir_path : ", dir_path)
        return False

In [4]:
wd = webdriver.Chrome(chrome_path)
wd.get(pendant)

In [5]:
event = wd.find_element_by_link_text("SEARCH")
event.click()
blank_search = wd.find_elements_by_name("search_query")
# blank_search[1] and blank_search[2] is interactable

blank_search[1].send_keys("") # test whether ele is interactable
blank_search[1].send_keys(Keys.RETURN) # go to https://www.slader.com/search/?search_query=

search_ele = wd.find_element_by_id("search-input") # find the search column
for book in book_list:
    search_ele.clear()
    search_ele.send_keys(book)
    time.sleep(1) # wait html change after send_keys
    soup = bs(search_ele.find_element_by_xpath("//body").get_attribute('outerHTML'))
    
    # get all books relative to keywords in book_list
    for ele in soup.select(".textbook-work-index-widget"):
        book_url.append(pendant + ele["data-url"])
    if len(book_url) == 0:
        print("did not get any url, please check your keywords")

In [ ]:
for ele in book_url:
    wd.get(ele)
    time.sleep(1)
    page_input = wd.find_element_by_xpath("//form[@class='textbook-page-search']/input[1]") # find page input column
    
    # go to the first page
    page_input.send_keys("1") 
    page_input.send_keys(Keys.RETURN)
    
    # use request to get first_page (faster than selenium)
    first_page_url = wd.current_url
    try:
        first_page_content = requests.get(first_page_url)
    except:
        print("requests error in : " + first_page_url)
    first_page_soup = bs(first_page_content.text, "html.parser")
    
    # find name of this book
    book_name = first_page_soup.find("a", {"class":"textbook desktop"}).get_text().strip()
    ISBN = first_page_soup.find("section", {"class":"bottom-row"}).get_text().strip().replace(" ","").replace(":","").replace("/","_")
    print(book_name," ",ISBN)
    
    # find all pages have excercises in first page
    section_menu = first_page_soup.find_all("option",{"class":"section-menu-item"})
    
    # request all pages (include first page)
    for page_data in section_menu:
        page_url = pendant + page_data["data-url"] # this page url
        try:
            page_content = requests.get(page_url)
        except:
            print("requests error in : " + page_url)
        page_soup = bs(page_content.text, "html.parser")
        excercise = page_soup.find_all("div",{"class":"list-item unbound exercise-in-group-item"}) # excercise in this page
        
        # find_page_num
        page_num = "page_" + page_soup.select(".textbook-page-search input")[0]["value"]
        
        # handle all excercises in this page
        for ex_url in excercise:
            wd.get(pendant + ex_url["data-url"]) # use selenium to go to each excercise
            time.sleep(1)
            
            ex_soup = bs(wd.page_source)
            ex_str = ex_soup.find_all("a", {"class":"bc"})[-1].get_text().strip().replace(" ", "")
            
            # get img of excercise and download it
            img_num = 0
            for ele in ex_soup.select(".solution-content img"):
                img_url = ele["src"]
                img_name = book_name + "_" + ex_str + str(img_num)
                store_path = "{}{}{}{}{}{}{}{}".format(file_path, os.sep, book_name, ISBN, os.sep, page_num, os.sep, ex_str)
                check = download_img(store_path, img_name, img_url)
                if check:
                    img_num += 1
            
    time.sleep(1)